# 사용자 정의 지표 만들기  
<p>케라스에서 모델의 성능을 평가하는 지표를 사용자가 직접 정의할 수 있다.</p>
<p>IMDB 데이터셋으로 영화 리뷰가 긍정적인지 부정적인지를 판별하는 이진 분류 모델을 만들고,</p>
<p>직접 정의한 지표를 통해 모델의 학습 과정을 평가해보도록 하자.</p>

In [1]:
# 필요한 라이브러리 불러오기
from tensorflow.keras.datasets import imdb
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
import numpy as np

In [2]:
# 데이터 로드 및 전처리
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results
train_data = vectorize_sequences(train_data)

In [3]:
train_labels

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [4]:
# 모델 구축하기
def build_model():
    model = keras.Sequential([
        layers.Dense(8, activation="relu"),
        layers.Dense(8, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])
    return model

### 사용자 정의 지표 만들기

<p>클래스를 만들고, keras.metrics.Metric을 상속하여 만든다.</p>

<p>아래 코드 블럭 세 개는 각각 accuracy, Precision, Recall을 직접 정의한 것이다.</p>

In [5]:
# Accuracy 클래스 정의하기
class MyAccuracy(keras.metrics.Metric):
    def __init__(self, name="my_accuracy", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(
            name="true_positives", initializer="zeros")
        self.true_negatives = self.add_weight(
            name="true_negatives", initializer="zeros")
        self.false_positives = self.add_weight(
            name="false_positives", initializer="zeros")
        self.false_negatives = self.add_weight(
            name="false_negatives", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(tf.cast(y_true, tf.float32))
        y_pred = tf.squeeze(tf.cast(y_pred >= 0.5, tf.float32))

        true_pos = tf.reduce_sum(y_true * y_pred)
        true_neg = tf.reduce_sum((1 - y_true) * (1 - y_pred))
        false_pos = tf.reduce_sum((1 - y_true) * y_pred)
        false_neg = tf.reduce_sum(y_true * (1 - y_pred))

        self.true_positives.assign_add(true_pos)
        self.true_negatives.assign_add(true_neg)
        self.false_positives.assign_add(false_pos)
        self.false_negatives.assign_add(false_neg)
                                
    def result(self):
        # 정확도 계산
        return  ((self.true_positives) + (self.true_negatives)) / (self.true_positives + self.true_negatives + self.false_positives + self.false_negatives)           
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.true_negatives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)

In [6]:
# Precision 클래스 정의하기
class MyPrecision(keras.metrics.Metric):
    def __init__(self, name="my_precision", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="true_positives", initializer="zeros")
        self.predicted_positives = self.add_weight(name="predicted_positives", initializer="zeros") 

    def update_state(self, y_true, y_pred, sample_weight=None):
        # For binary classification, threshold predictions at 0.5
        y_pred_classes = tf.squeeze(tf.cast(y_pred >= 0.5, "float32"))
        y_true = tf.squeeze(tf.cast(y_true, "float32"))
        
        true_positives = tf.reduce_sum(y_true * y_pred_classes)
        predicted_positives = tf.reduce_sum(y_pred_classes)
        
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)     
        
    def result(self):
        # Precision 계산
        return  (self.true_positives)/(self.predicted_positives + tf.keras.backend.epsilon())
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.predicted_positives.assign(0)

In [7]:
# Precision 클래스 정의하기
class MyRecall(keras.metrics.Metric):
    def __init__(self, name="my_recall", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="true_positives", initializer="zeros")
        self.actual_positives = self.add_weight(name="actual_positives", initializer="zeros") 

    def update_state(self, y_true, y_pred, sample_weight=None):
        # For binary classification, threshold predictions at 0.5
        y_pred_classes =  tf.squeeze(tf.cast(y_pred >= 0.5, "float32"))
        y_true = tf.squeeze(tf.cast(y_true, "float32"))
        
        true_positives = tf.reduce_sum(y_true * y_pred_classes)
        actual_positives = tf.reduce_sum(y_true)
        
        self.true_positives.assign_add(true_positives)
        self.actual_positives.assign_add(actual_positives)     
        
    def result(self):
        # Recall 계산
        return  (self.true_positives)/(self.actual_positives + tf.keras.backend.epsilon())
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.actual_positives.assign(0)

In [8]:
# 모델 학습
model_new = build_model()
model_new.compile(optimizer="rmsprop",
              loss="binary_crossentropy",

              # 'accuracy'는 기본으로 제공되는 지표이다.
              # 'MyAccuracy()', 'MyPrecision()', 'MyRecall()'은 위 블럭에서 직접 정의한 지표이다.
              # 'accuracy'값과 'MyAccuracy()'값이 동일하게 나오는 것을 확인할 수 있다.
              metrics=['accuracy', MyAccuracy(), MyPrecision(), MyRecall()]) 
history_original = model_new.fit(train_data, train_labels,
                             epochs=5, batch_size=16, validation_split=0.4)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8068 - loss: 0.4370 - my_accuracy: 0.8068 - my_precision: 0.7961 - my_recall: 0.8229 - val_accuracy: 0.8880 - val_loss: 0.2858 - val_my_accuracy: 0.8880 - val_my_precision: 0.8550 - val_my_recall: 0.9328
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9208 - loss: 0.2045 - my_accuracy: 0.9208 - my_precision: 0.9156 - my_recall: 0.9270 - val_accuracy: 0.8889 - val_loss: 0.2884 - val_my_accuracy: 0.8889 - val_my_precision: 0.8961 - val_my_recall: 0.8782
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9433 - loss: 0.1584 - my_accuracy: 0.9433 - my_precision: 0.9421 - my_recall: 0.9440 - val_accuracy: 0.8867 - val_loss: 0.3115 - val_my_accuracy: 0.8867 - val_my_precision: 0.9026 - val_my_recall: 0.8653
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9533 - loss: 0.1306 - my_accuracy: 0.9533 - my_precision: 0.9526 - my_recall: 0.9546 - val_accuracy: 0.8861 - val_loss: 0.3304 